In [1]:
import yaml
import numpy as np
import matplotlib.pyplot as plt
from os import listdir

In [ ]:
# Fit folder
FitFolders = ["/Users/s2569857/Codes/NTK-interface/build/run/Results/test",]

In [12]:
from abc import ABC, abstractmethod

_data_sample_size = 4

class Observable(ABC):

    def __init__(self, FitFolder, shape, observable_name):
        self.FitFolder = FitFolder
        self.number_of_replicas = len(listdir(FitFolder + "/log/"))
        self.shape = shape
        self.observable_name = observable_name

        with open(FitFolder + "/meta.yaml", "r") as meta:
            Meta = yaml.load(meta, Loader=yaml.CLoader)

        Meta["data_sample_size"] = _data_sample_size

        self.size_of_data = Meta["data_sample_size"]
        self.size_of_output = Meta['NNarchitecture'][-1]
        self.size_of_NN = Meta["NNarchitecture"][1]

    def load_observable(self, replica, tr_step):
        with open(self.FitFolder + "/log/replica_" + str(replica) + ".yaml", "r") as rep:
            Rep = yaml.load(rep, Loader=yaml.CLoader)
        out = np.ndarray(shape=self.shape, dtype=float, order = 'C', buffer=np.array(Rep[tr_step][self.observable_name]))
        return out
    
    def evaluate(self, replica, tr_step):
        return self.load_observable(replica, tr_step)
        
    def average_over_replicas(self, tr_step):

        shape = self.shape
        obs = np.zeros(shape, dtype=float)
        observable_by_replica = np.array([self.evaluate(frep, tr_step) for frep in range(1, self.number_of_replicas)])
        obs = np.average(observable_by_replica, axis = 0)
        std = np.std(observable_by_replica, axis = 0)

        return obs, std, observable_by_replica

In [13]:
class NTK_element_wise(Observable):
    def __init__ (self, FitFolder, shape, observable_name):
        super().__init__(FitFolder, shape, observable_name)

class NTK_frob(Observable):
    def __init__ (self, FitFolder, shape, observable_name):
        super().__init__(FitFolder, shape, observable_name)
    
    def evaluate(self, replica, tr_step):
        return np.linalg.norm(self.load_observable(replica, tr_step))
    
class O3(Observable):
    def __init__ (self, FitFolder, shape, observable_name):
        super().__init__(FitFolder, shape, observable_name)

## Loop over FitFolders and plot a grapgh as function of the size of the network
The plots are generated at initialisation

In [ ]:
labels = np.zeros(len(FitFolders))
ntk_mean_values = np.zeros(len(FitFolders))
ntk_std_values = np.zeros(len(FitFolders))
O3_mean_values = np.zeros(len(FitFolders))
O3_std_values = np.zeros(len(FitFolders))

for i, fit in enumerate(FitFolders):
    

    # TO_DO
    # _data_sample_size is hardcoded and not really recommended. Try to
    # to make it more general
    ntk_frob = NTK_frob(fit, shape=(_data_sample_size, _data_sample_size), observable_name="NTK_eigen")
    o3 = O3(fit, shape=(_data_sample_size, _data_sample_size, _data_sample_size), observable_name="O_3")
    ntk_frob_mean, ntk_frob_std = ntk_frob.average_over_replicas(0)
    o3_mean, o3_std = o3.average_over_replicas(0)

    # Choose a particular data combination for O_3
    o3_mean_data = o3_mean[0,1,2]
    o3_std = o3_std[0,1,2]



In [14]:
ntk = NTK_element_wise(FitFolders[0], shape=(_data_sample_size, _data_sample_size), observable_name="NTK_eigen")




In [30]:
obs_mean, _, obs_vec = o3.average_over_replicas(0)